# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [80]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
import gmaps
import os
import json
from holoviews import opts
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [81]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = "output_data/cities.csv"
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sinop,41.7711,34.8709,16.71,82,0,1.47,TR,1690852129
1,1,waitangi,-43.9535,-176.5597,13.79,87,1,2.68,NZ,1690851974
2,2,mount isa,-20.7333,139.5000,23.87,35,100,4.12,AU,1690852131
3,3,puerto ayora,-0.7393,-90.3518,24.75,93,57,1.79,EC,1690852132
4,4,port-aux-francais,-49.3500,70.2167,5.18,93,100,18.76,TF,1690852133


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [82]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color= "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [83]:
# Filter vacation with zero cloudiness
vacation_no_cloud = city_data_df[city_data_df["Cloudiness"] == 0]
# Filter vacation with max temp above 25 degrees C
vacation_above_25_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp"] > 25]
# Filter vacation with max temp below 30 degrees C
vacation_below_30_degrees = vacation_above_25_degrees[vacation_above_25_degrees["Max Temp"] < 30]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_30_degrees[vacation_below_30_degrees["Wind Speed"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity"] < 60]
# Set Index
index_perfect_vacation = perfect_vacation.reset_index()
del index_perfect_vacation["index"]
index_perfect_vacation

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,24,argo,37.6333,22.7333,26.55,54,0,1.02,GR,1690852145
1,80,tamanrasset,22.7850,5.5228,28.95,10,0,2.06,DZ,1690852177
2,155,urumqi,43.8010,87.6005,26.53,28,0,2.00,CN,1690852086
3,245,minot,48.2325,-101.2963,29.12,54,0,3.60,US,1690852274
4,282,portland,45.5234,-122.6762,29.74,35,0,5.66,US,1690852185
5,288,ravar,31.2656,56.8055,26.01,15,0,2.50,IR,1690852301
6,391,yorkton,51.2167,-102.4677,27.91,47,0,4.63,CA,1690852361
7,427,at taj,24.2049,23.2857,28.45,28,0,3.71,LY,1690852380
8,447,fort mcmurray,56.7268,-111.3810,25.77,44,0,7.20,CA,1690852392
9,514,puerto quijarro,-17.7833,-57.7667,25.84,31,0,1.02,BO,1690852432


### Step 3: Create a new DataFrame called `hotel_df`.

In [84]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = index_perfect_vacation.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,24,argo,37.6333,22.7333,26.55,54,0,1.02,GR,1690852145,
1,80,tamanrasset,22.7850,5.5228,28.95,10,0,2.06,DZ,1690852177,
2,155,urumqi,43.8010,87.6005,26.53,28,0,2.00,CN,1690852086,
3,245,minot,48.2325,-101.2963,29.12,54,0,3.60,US,1690852274,
4,282,portland,45.5234,-122.6762,29.74,35,0,5.66,US,1690852185,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [85]:
# Set parameters to search for a hotel
limit = 20
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},{radius}",
        "limit": limit,
        "bias": f"proximity:{lng},{lat}",
        "apiKey": geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
argo - nearest hotel: Apollon Hotel
tamanrasset - nearest hotel: فندق الأمان
urumqi - nearest hotel: 百时快捷酒店红山店
minot - nearest hotel: Hampton Inn & Suites
portland - nearest hotel: the Hoxton
ravar - nearest hotel: زائر سرا راور
yorkton - nearest hotel: Ramada Yorkton
at taj - nearest hotel: No hotel found
fort mcmurray - nearest hotel: Nomad Hotel and Suites
puerto quijarro - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,24,argo,37.6333,22.7333,26.55,54,0,1.02,GR,1690852145,Apollon Hotel
1,80,tamanrasset,22.7850,5.5228,28.95,10,0,2.06,DZ,1690852177,فندق الأمان
2,155,urumqi,43.8010,87.6005,26.53,28,0,2.00,CN,1690852086,百时快捷酒店红山店
3,245,minot,48.2325,-101.2963,29.12,54,0,3.60,US,1690852274,Hampton Inn & Suites
4,282,portland,45.5234,-122.6762,29.74,35,0,5.66,US,1690852185,the Hoxton
5,288,ravar,31.2656,56.8055,26.01,15,0,2.50,IR,1690852301,زائر سرا راور
6,391,yorkton,51.2167,-102.4677,27.91,47,0,4.63,CA,1690852361,Ramada Yorkton
7,427,at taj,24.2049,23.2857,28.45,28,0,3.71,LY,1690852380,No hotel found
8,447,fort mcmurray,56.7268,-111.3810,25.77,44,0,7.20,CA,1690852392,Nomad Hotel and Suites
9,514,puerto quijarro,-17.7833,-57.7667,25.84,31,0,1.02,BO,1690852432,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [86]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color= "City",
    hover_cols= ["Hotel Name", "Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)